# Experiment 1: Fine-Tuning / From Scratch Training

In [2]:
import pandas as pd
import pyvista as pv
%pip install trame

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.




[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
results = pd.read_csv("../eval/TestEvaluation/mesh_eval_results.csv")

# print all models unqieu in csv

re_models = results["model"].unique()
print(re_models)

['pix2vox_aug' 'pix2vox_aug_1img' 'pix2vox_aug_3img' 'pix2vox_aug_4img'
 'pix2vox_aug_5img' 'pix2vox_aug_6img' 'pix2vox_aug_frozen_encdec'
 'pix2vox_base' 'pix2vox_non_pretrained' 'vh_1img' 'vh_2img' 'vh_3img'
 'vh_4img' 'vh_5img' 'vh_6img' 'coarse_mvp2m_2_inputs_none_augmentation'
 'coarse_mvp2m_augmentation_2_inputs'
 'coarse_mvp2m_augmentation_2_inputs_freezed'
 'coarse_mvp2m_augmentation_2_inputs_pretrained'
 'coarse_mvp2m_augmentation_2_inputs_use_mean_prior'
 'coarse_mvp2m_augmentation_2_inputs_use_spherical_prior'
 'coarse_mvp2m_augmentation_3_inputs' 'coarse_mvp2m_augmentation_4_inputs'
 'coarse_mvp2m_augmentation_5_inputs' 'coarse_mvp2m_augmentation_6_inputs'
 'Hunyuan3D' 'Hunyuan3D-four-views' 'Hunyuan3D-two-views'
 'Pixel2Mesh_Augmented_2_Views' 'pollen_128_2' 'pollen_256_2'
 'pollen_augmentation1' 'pollen_augmentation2' 'pollen_augmentation3'
 'pollen_augmentation4' 'pollen_augmentation6'
 'pollen_augmentation_finetune' 'pollen_augmentation_freezed'
 'pollen_augmentation_no

In [7]:
relevant_models = [
    "vh_2img",   
    "pix2vox_aug",
    "pix2vox_aug_frozen_encdec",
    "pix2vox_non_pretrained",
    "pollen_augmentation_finetune",
    "pollen_augmentation_freezed",
    "pollen_none_pretrained",
    "refine_p2mpp_augmentation_2_inputs",
    "refine_p2mpp_augmentation_2_inputs_freezed",
    "refine_p2mpp_augmentation_2_inputs_pretrained",
]

filtered = results[results["model"].isin(relevant_models)]

aggregated = filtered.groupby("model", as_index=False).agg(
    mean_chamfer         = ("chamfer",     "mean"),
    std_chamfer          = ("chamfer",     "std"),
    mean_fscore_1        = ("fscore_1",    "mean"),
    std_fscore_1         = ("fscore_1",    "std"),
    mean_fscore_2_5      = ("fscore_2_5",  "mean"),
    std_fscore_2_5       = ("fscore_2_5",  "std"),
    mean_fscore_5        = ("fscore_5",    "mean"),
    std_fscore_5         = ("fscore_5",    "std"),
    mean_iou             = ("voxel_iou",   "mean"),
    std_iou              = ("voxel_iou",   "std"),
)

aggregated

,model,mean_chamfer,std_chamfer,mean_fscore_1,std_fscore_1,mean_fscore_2_5,std_fscore_2_5,mean_fscore_5,std_fscore_5,mean_iou,std_iou
0,pix2vox_aug,0.059839,0.020741,0.166688,0.060798,0.603350,0.151200,0.848221,0.121432,0.713982,0.176484
1,pix2vox_aug_frozen_encdec,0.056526,0.017044,0.180305,0.063962,0.619251,0.149610,0.850548,0.099744,0.678091,0.139773
2,pix2vox_non_pretrained,0.063499,0.018741,0.148749,0.046822,0.568961,0.138251,0.819073,0.118238,0.722673,0.109763
3,pollen_augmentation_finetune,0.044596,0.016757,0.257467,0.088129,0.755981,0.152617,0.904351,0.096696,0.823766,0.071286
4,pollen_augmentation_freezed,0.059333,0.015406,0.176354,0.065748,0.623380,0.092378,0.827504,0.097487,0.792415,0.077769
5,pollen_none_pretrained,0.044786,0.018491,0.261999,0.114809,0.759020,0.165802,0.900828,0.106800,0.825780,0.083049
6,refine_p2mpp_augmentation_2_inputs,0.051909,0.022576,0.212900,0.090377,0.690881,0.155863,0.884539,0.124626,0.406086,0.234189
7,refine_p2mpp_augmentation_2_inputs_freezed,0.051375,0.025330,0.216349,0.091418,0.687749,0.159364,0.886603,0.139626,0.763154,0.117579
8,refine_p2mpp_augmentation_2_inputs_pretrained,0.052216,0.021743,0.216471,0.096116,0.687208,0.154837,0.879575,0.119737,0.769657,0.110282
9,vh_2img,0.091423,0.024904,0.109239,0.040237,0.416278,0.101777,0.664201,0.120545,0.637425,0.090634


In [5]:
mesh1 = pv.read("../TestEvaluationPipeline/data/pix2vox_aug/17781_Common_thyme_Thymus_vulgaris_pollen_grain.stl")
mesh2 = pv.read("../TestEvaluationPipeline/data/pix2vox_aug/17781_Common_thyme_Thymus_vulgaris_pollen_grain.stl")
mesh3 = pv.read("../TestEvaluationPipeline/data/pix2vox_aug/17781_Common_thyme_Thymus_vulgaris_pollen_grain.stl")
mesh4 = pv.read("../TestEvaluationPipeline/data/pix2vox_aug/17781_Common_thyme_Thymus_vulgaris_pollen_grain.stl")

# Suppose meshes is a list of pyvista PolyData objects
meshes = [mesh1, mesh2, mesh3, mesh4]

# Create a 2×2 grid of subplots
plotter = pv.Plotter(shape=(2, 2), off_screen=True)

for idx, mesh in enumerate(meshes):
    i, j = divmod(idx, 2)
    plotter.subplot(i, j)
    plotter.add_text(f"Mesh {idx+1}", font_size=12)
    plotter.add_mesh(mesh, show_edges=True)

# Optionally link camera views across subplots
plotter.link_views()

# Render and save screenshot
plotter.show(screenshot="mesh_grid.png", window_size=(800, 800))

FileNotFoundError: File (C:\Users\super\Documents\Github\sequoia\TestEvaluationPipeline\data\pix2vox_aug\17781_Common_thyme_Thymus_vulgaris_pollen_grain.stl) not found